In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType, DateType
import sys
import os
from delta import DeltaTable



from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
from delta.tables import *
import io
import json

In [ ]:
def create_spark_session():
    return SparkSession \
        .builder \
        .appName("File Streaming Demo") \
        .master("local[3]") \
        .config("spark.databricks.delta.schema.autoMerge.enabled", "true")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .enableHiveSupport()\
        .getOrCreate()

In [ ]:
spark = create_spark_session()

In [ ]:
schema = StructType([ 
    StructField("product_id",IntegerType(),True), \
    StructField("product_name",StringType(),True),
    StructField("product_category_name",StringType(),True), \
    StructField("product_price", DoubleType(), True) 
  ])

data_product = [(1,'perfurme magico','perfumaria',100.5),
                (2,'replica vaso ming','artes',55.75),
               (3,'raquete de tenis','esporte_lazer',350.00),
                (4,'mordedor','bebes',27.25),
                (5,'televisor 46 polegadas 4k','utilidades_domesticas',2555.55)
               ]


df_products = spark.createDataFrame(data=data_product,schema=schema)



In [ ]:
df_products.display()

product_id,product_name,product_category_name,product_price
1,perfurme magico,perfumaria,100.5
2,replica vaso ming,artes,55.75
3,raquete de tenis,esporte_lazer,350.0
4,mordedor,bebes,27.25
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55


### Criando delta table utilizando write
A forma mais simples de criar uma Delta Table informando o format "delta", como abaixo. <br>
Abaixo uma Delta Table é criada e os dados podem ser sobrescritos ou acrescentados, <br> 
utilizando respectivamente as opções
mode **OVERWRITE** ou **APPEND** </br>
É importante frisar que utilizando um spark dataframe, só temos acesso a acrescentar ou adicionar dados num repositório previamente escrito.

In [ ]:
dbutils.fs.rm('/FileStore/bronze/aula_delta/products',True)

Out[35]: False

In [ ]:
path_products = '/FileStore/bronze/aula_delta/products'
mode = 'OVERWRITE'
df_products\
.write.format("delta")\
.option("overwriteSchema", "true") \
.save(path_products, mode=mode)

### Carregando os dados escritos como delta table 
Para a utilização dos metodos do Delta Table, os dados devem ser carregados como Delta Tables <br>
atravez do método **DeltaTable.forPath(spark, caminho_Delta_table)**

**Importante** Uma DeltaTable não é um spark dataframe, então caso seja necessário utilizar operações e ações de um <br> 
spark dataframe existem duas formas :
* Carregar os dados gravados no formato delta para um dataframe pyspark,utilizar o spark.read.format('delta').load(caminho_Delta_table) 

* Carregar uma DeltaTable com **DeltaTable.forPath(spark, caminho_Delta_table)** e usar o metodo **toDf()** , que retorna um dataframe spark de uma tabela delta

In [ ]:
path_products = '/FileStore/bronze/aula_delta/products'
df_products_read = spark.read.format('delta').load(path_products)
df_products_read.display()

product_id,product_name,product_category_name,product_price
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55
4,mordedor,bebes,27.25


In [ ]:
path_products = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, path_products)
df_from_delta = deltaTable.toDF()
df_from_delta.display()

product_id,product_name,product_category_name,product_price
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55
4,mordedor,bebes,27.25


### Apagando dados delta table 
Com DeltaTables é possível remover dados a nivel de linha, sem ser necessário sobrescrever os dados . <br>
Carregue uma DeltaTable com **DeltaTable.forPath(spark, location_tb)** e utilize o método delete para informar a <br> 
condição de remoção. <br>
Uma String com uma condição SQL pode ser utilizada ou a notação spark.



In [ ]:
# condição no formato sql.
deltaTable.delete("product_id = 5")

# condição no formato spark
deltaTable.delete(col('product_id') == 4)

In [ ]:
df_from_delta.display(truncate = False)

product_id,product_name,product_category_name,product_price
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55


### Alterando dados delta table 
Com DeltaTables é possível alterar dados a nivel de linha, sem ser necessário sobrescrever os dados . <br>
Carregue uma DeltaTable com **DeltaTable.forPath(spark, location_tb)** e utilize o update delete para informar a <br> 
condição de remoção. <br>
Uma String com uma condição SQL pode ser utilizada ou a notação spark. <br>
O método update utiliza dois parametros, um para a condição de update e outro para informar quais campos serão alterados <br> e como serão alterados.


In [ ]:
# condição no formato sql.
deltaTable.update(
  condition = "product_id = 1",
  set = { "product_name": "'perfurme magico cheiroso'" }
)



In [ ]:
# condição formato spark , utilizando uma função spark para informar o nome da coluna
deltaTable.update(
  condition = col('product_id') == 1,
  set = { 'product_price': col('product_price') * 1.2 }
)

In [ ]:
df_from_delta.display()

product_id,product_name,product_category_name,product_price
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55


### Utilizando MERGE para inserir, alterar e remover dados de uma Delta Table
Outra forma de atualizar, remover e inserir dados numa DeltaTable é com **MERGE** <br>
Um metodo MERGE abaixo se assemelha ao exemplo abaixo <br>

<br>
deltaTable.alias('tgt') \ <br>
.merge( <br>
    df_products.alias('src'), <br>
    "tgt.product_id = src.product_id" <br>
) \ <br> 
.whenMatchedUpdateAll() \ <br>
.whenNotMatchedInsertAll() \ <br>
.execute() <br>
<br>

O metodo merge atualiza uma DeltaTable existente utilizando dados de um spark dataframe, onde 
o DeltaTable é comparado com o spark dataframe utilizando uma coluna que funciona como uma **Chave Unica (Sem repetição)** ,
apartir do resultado da comparação eventos de inserção e atualização são disparados. Os eventos mais comuns são : <br>
* **whenMatchedUpdateAll()** quando a chave for encontrada, todas os campos das linhas que atendam a condição da DeltaTable são atualizadas com o conteudo do dataframe spark <br>
* **whenNotMatchedInsertAll()** quando a chave não for encontrada, todas as linhas que atendam a condição do dataframe spark são inseridas no DeltaTable <br> 
* **whenMatchedUpdate**  quando a chave for encontrada, os campos informados das linhas que atendam a condição da DeltaTable são atualizadas com o conteudo do dataframe spark<br>
* **whenNotMatchedInsert** quando a chave não for encontrada, Os campos informados de todas as linhas que atendam a condição do dataframe spark são inseridas no DeltaTable <br> 
* **whenMatchedDelete** quando a condição for satisfeita, todas as linhas que atendam a condição são removidas do DeltaTable.


In [ ]:
schema = StructType([ 
    StructField("product_id",IntegerType(),True), \
    StructField("product_name",StringType(),True),
    StructField("product_category_name",StringType(),True), \
    StructField("product_price", DoubleType(), True) 
  ])

data_product = [(2,'replica vaso ming importado','artes',70.75),
                (3,'raquete de tenis nacional','esporte_lazer',365.00),
                (4,'mordedor','bebes',27.25),
                (5,'televisor 46 polegadas 4k','utilidades_domesticas',2555.55),
                (6,'PS','utilidades_domesticas',2555.55)
               ]


df_products = spark.createDataFrame(data=data_product,schema=schema)


In [ ]:
df_products.display()

product_id,product_name,product_category_name,product_price
2,replica vaso ming importado,artes,70.75
3,raquete de tenis nacional,esporte_lazer,365.0
4,mordedor,bebes,27.25
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55
6,PS,utilidades_domesticas,2555.55


In [ ]:
df_from_delta.display()

product_id,product_name,product_category_name,product_price
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55


Abaixo o método MERGE é utilizado numa DeltaTable, onde a inserção e atualização é realizada somente com os campos especificos </br>
Uma operação de MERGE é realizada entre uma deltaTable e um dataframe spark, onde : </br>
* **deltaTable()** É o dado já gravado no datalakehouse, onde serão inseridos ou atualizados os registros escritos anteriormente </br>
* **dataframe spark()** É o novo dado, contendo os novos registros ou registros modificados</br> 


In [ ]:
deltaTable.alias("target")\
.merge(
    source = df_products.alias("source"),
    condition = "source.product_id = target.product_id"
).whenMatchedUpdate(
    set =
        {
          "target.product_name": "source.product_name",
          "target.product_category_name": "source.product_category_name"
        }
).whenNotMatchedInsert(
    values =
        {          
          "target.product_name": "source.product_price",
          "target.product_category_name": "source.product_category_name",
          "target.product_name": "source.product_name",
          "target.product_price": "source.product_price"
        }
).execute()

In [ ]:
df_from_delta.display()

product_id,product_name,product_category_name,product_price
null,televisor 46 polegadas 4k,utilidades_domesticas,2555.55
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55
null,mordedor,bebes,27.25


No utiliza ção anterior, durante o metodo **whenNotMatchedInsert**, o campo product_id não foi informando, resultado <br>na inserção com o valor nullo na DeltaTable, como esse campo é utilizado como chave, erros podem ocorrer, o ideal é remover a linha. <br>
Abaixo o método **MERGE** é utilizado numa DeltaTable em conjunto com **whenMatchedDelete**, para remover as linhas onde o campo product_id seja null

In [ ]:
### Usando merge com delete
deltaTable.alias("target")\
.merge(
    source = df_products.alias("source"),
    condition = "target.product_id is null"
).whenMatchedDelete()\
.execute()

In [ ]:
df_from_delta.display()

product_id,product_name,product_category_name,product_price
3,raquete de tenis nacional,esporte_lazer,365.0
2,replica vaso ming importado,artes,70.75
6,PS,utilidades_domesticas,2555.55


Abaixo o método **MERGE** é utilizado numa DeltaTable em conjunto com **whenMatchedUpdateAll** e **whenNotMatchedInsertAll**, para atualizar todas as linhas encontradas no dataframe e DeltaTable ou inserir todas as linhas não encontradas <br>
Os dois metodos consideram que todos os campos são atualizados ou inseridos.

In [ ]:
merge_condition = "tgt.product_id = src.product_id"
deltaTable.alias('tgt') \
.merge(
    df_products.alias('src'),
    merge_condition
) \
.whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

In [ ]:
df_from_delta.display()

product_id,product_name,product_category_name,product_price,status
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55,Active
3,raquete de tenis nacional,esporte_lazer,365.0,Active
2,replica vaso ming importado,artes,70.75,Active
6,PS,utilidades_domesticas,2555.55,Active
4,mordedor,bebes,27.25,Inative


### Modificando o Schema de uma DeltaTable existente
É possivel modificar o schema de uma DeltaTable existente, adicionando a configuração <br> **config("spark.databricks.delta.schema.autoMerge.enabled", "true")** na sparkSession. <br>
No exemplo abaixo, a nova coluna **status** foi adicionado ao Dataframe, para os registros antigos, o valor do campo será null .


In [ ]:
schema = StructType([ 
    StructField("product_id",IntegerType(),True), \
    StructField("product_name",StringType(),True),
    StructField("product_category_name",StringType(),True), \
    StructField("product_price", DoubleType(), True),
    StructField("status",StringType(),True),
  ])

data_product = [(2,'replica vaso ming importado','artes',70.75,'Active'),
                (3,'raquete de tenis nacional','esporte_lazer',365.00,'Active'),
                (4,'mordedor','bebes',27.25,'Inative'),
                (5,'televisor 46 polegadas 4k','utilidades_domesticas',2555.55,'Active'),
                (6,'PS','utilidades_domesticas',2555.55,'Active')
               ]


df_products = spark.createDataFrame(data=data_product,schema=schema)


In [ ]:
df_products.display()

product_id,product_name,product_category_name,product_price,status
2,replica vaso ming importado,artes,70.75,Active
3,raquete de tenis nacional,esporte_lazer,365.0,Active
4,mordedor,bebes,27.25,Inative
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55,Active
6,PS,utilidades_domesticas,2555.55,Active


In [ ]:
path_products = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, path_products)
merge_condition = "tgt.product_id = src.product_id"
deltaTable.alias('tgt') \
.merge(
    df_products.alias('src'),
    merge_condition
) \
.whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

In [ ]:
path_products = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, path_products)
df_from_delta = deltaTable.toDF()
df_from_delta.display()


product_id,product_name,product_category_name,product_price,status
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55,Active
3,raquete de tenis nacional,esporte_lazer,365.0,Active
2,replica vaso ming importado,artes,70.75,Active
6,PS,utilidades_domesticas,2555.55,Active
4,mordedor,bebes,27.25,Inative


### Criando uma DeltaTable utilizando create
Outra forma de criar uma DeltaTable é utilizando o metodo **create** , onde os campos e local de escrita são informados.
O método create cria a DeltaTable vazia, possibilitando a utilização de métodos como o **merge** , **update** e **delete** . <br>
Essa é uma forma de criação bastante interessante para a criação de métodos inserção e atualização mais genéricos. <br>


Abaixo o codigo realiza as tarefas
* criando a lista das colunas utilizadas
* verifica se existe uma DeltaTable no diretório informado, caso positivo, executa um MERGE para atualização/inserção
* verifica se existe uma DeltaTable no diretório informado, caso negativo, cria uma DeltaTable com a estrutura informada e executa um MERGE para atualização/inserção

In [ ]:
location_tb = '/FileStore/bronze/aula_delta/products'
dbutils.fs.rm(location_tb,True)

Out[81]: True

In [ ]:

location_tb = '/FileStore/bronze/aula_delta/products'
merge_condition = "tgt.product_id = src.product_id"

columns = [
    StructField('product_id', IntegerType(), True),
    StructField('product_name', StringType(), True),
    StructField('product_category_name', StringType(), True),
    StructField('product_price', DoubleType(), True),
    StructField('status', StringType(), True)
]

if (DeltaTable.isDeltaTable(spark, location_tb)):
    print('tabela delta existente')
    deltaTable = DeltaTable.forPath(spark, location_tb)
    deltaTable.alias('tgt') \
        .merge(
            df_products.alias('src'),
            merge_condition
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    print('tabela delta inexistente')    
    DeltaTable \
            .create(spark) \
            .addColumns(columns) \
            .location(location_tb) \
            .execute()
    deltaTable = DeltaTable.forPath(spark, location_tb)
    deltaTable.alias('tgt') \
        .merge(
            df_products.alias('src'),
            merge_condition
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()


tabela delta inexistente


In [ ]:
path_products = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, path_products)
df_from_delta = deltaTable.toDF()
df_from_delta.display()

product_id,product_name,product_category_name,product_price,status
5,televisor 46 polegadas 4k,utilidades_domesticas,2555.55,Active
3,raquete de tenis nacional,esporte_lazer,365.0,Active
2,replica vaso ming importado,artes,70.75,Active
6,PS,utilidades_domesticas,2555.55,Active
4,mordedor,bebes,27.25,Inative
